### Realizado por:
1. John Anderson Acosta 202212004

En primer lugar, Cargamos el conjunto de datos de Apple (aapl_features_P3.csv) y luego se selecciona las 8 características las etiquetas, y construimos la ventana de 120 días para el aprendizaje de series temporales.

In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv("aapl_features_P3.csv")

features = data.drop(columns=["y"]).values
labels = data["y"].values

window = 120

def create_windows(series, labels, window_size):
    X, y = [], []
    for i in range(len(series) - window_size):
        X.append(series[i:i + window_size])
        y.append(labels[i + window_size])
    return np.array(X), np.array(y)

X, y = create_windows(features, labels, window)
print(X.shape, y.shape)



(1078, 120, 8) (1078,)


Posteriormente, utilizamos el 80 % de los datos para el entrenamiento y el 20 % para las pruebas, manteniendo el orden temporal

In [2]:
split = int(len(X) * 0.8)
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]


Luego, se construyo el clasificador LSTM siguiendo la arquitectura requerida para los datos de Apple, en el examen fue de un tamaño oculto de 16, 2 capas y dropout de 0,3.

In [3]:
import torch
import torch.nn as nn

class LSTMClf(nn.Module):
    def __init__(self, input_size, hidden_size=16, num_layers=2, dropout=0.3):
        super().__init__()
        self.lstm = nn.LSTM(input_size=input_size,
                            hidden_size=hidden_size,
                            num_layers=num_layers,
                            dropout=dropout,
                            batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])
        return torch.sigmoid(out)


Luego, Entrenamos el modelo LSTM utilizando el optimizador AdamW y la pérdida de entropía cruzada binaria. La función devuelve la mejor precisión en todas las épocas.

In [4]:
import torch.optim as optim
from sklearn.metrics import accuracy_score

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def train_lstm(model, X_train, y_train, X_test, y_test, lr, weight_decay, epochs):
    model = model.to(device)
    criterion = nn.BCELoss()
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)

    Xtr = torch.tensor(X_train, dtype=torch.float32).to(device)
    ytr = torch.tensor(y_train.reshape(-1, 1), dtype=torch.float32).to(device)
    Xte = torch.tensor(X_test, dtype=torch.float32).to(device)
    yte = torch.tensor(y_test.reshape(-1, 1), dtype=torch.float32).to(device)

    best_acc, best_state = 0, None

    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        y_pred = model(Xtr)
        loss = criterion(y_pred, ytr)
        loss.backward()
        optimizer.step()

        if epoch % 100 == 0:
            model.eval()
            with torch.no_grad():
                preds = model(Xte)
                acc = accuracy_score(y_test, (preds.cpu().numpy() > 0.5).astype(int))
            if acc > best_acc:
                best_acc = acc
                best_state = model.state_dict()
            print(f"Epoch {epoch}: loss={loss.item():.4f}, acc={acc:.4f}")

    model.load_state_dict(best_state)
    return best_acc

model = LSTMClf(input_size=X.shape[2])
acc_test = train_lstm(model, X_train, y_train, X_test, y_test,
                      lr=5e-3, weight_decay=1e-5, epochs=1000)
print(f" Test Final del Accuracy (APPLE): {acc_test:.4f}")


Epoch 0: loss=0.6873, acc=0.5880
Epoch 100: loss=0.6603, acc=0.4954
Epoch 200: loss=0.6377, acc=0.3889
Epoch 300: loss=0.5993, acc=0.4630
Epoch 400: loss=0.6076, acc=0.3935
Epoch 500: loss=0.5646, acc=0.4306
Epoch 600: loss=0.5444, acc=0.4167
Epoch 700: loss=0.6053, acc=0.4120
Epoch 800: loss=0.6858, acc=0.4120
Epoch 900: loss=0.6081, acc=0.6065
Epoch 1000: loss=0.6036, acc=0.4120
 Test Final del Accuracy (APPLE): 0.6065


Al observar los resultados, durante el entrenamiento, la pérdida comenzó en 0,6873 y disminuyó gradualmente hasta alrededor de 0,60, mientras que la precisión osciló entre 0,41 y 0,60, estabilizándose finalmente en una precisión de prueba de 0,6065. Este comportamiento indica una convergencia parcial: el modelo fue capaz de reducir su pérdida con el tiempo y mejorar ligeramente la precisión predictiva, pero no logró una convergencia suave o fuerte. Estos resultados son esperables en los datos de series temporales financieras, donde la relación señal-ruido es baja y los rendimientos futuros solo están débilmente correlacionados con los indicadores pasados. el LSTM capturó con éxito algunas dependencias temporales dentro de las ventanas de entrada de 120 días, mostrando un progreso de aprendizaje modesto. Sin embargo, la alta volatilidad de la precisión a lo largo de las épocas sugiere que el modelo podría estar sobreajustadoo con overfitting[1][3]

Por último, entrenamos un segundo modelo utilizando únicamente la serie de precios para evaluar el valor añadido de incluir las otras variables.

In [5]:
X_price = data["price"].values
X_price_w, y_price = create_windows(X_price.reshape(-1,1), labels, window)

split = int(len(X_price_w) * 0.8)
Xtr_p, Xte_p = X_price_w[:split], X_price_w[split:]
ytr_p, yte_p = y_price[:split], y_price[split:]

model_price = LSTMClf(input_size=1)
acc_test_price = train_lstm(model_price, Xtr_p, ytr_p, Xte_p, yte_p,
                            lr=5e-3, weight_decay=1e-5, epochs=1000)

print(f"Test Final del accuracy usando solo el precio: {acc_test_price:.4f}")


Epoch 0: loss=0.7020, acc=0.4120
Epoch 100: loss=0.6703, acc=0.4120
Epoch 200: loss=0.6696, acc=0.4120
Epoch 300: loss=0.6801, acc=0.5880
Epoch 400: loss=0.6695, acc=0.4120
Epoch 500: loss=0.6596, acc=0.5880
Epoch 600: loss=0.6679, acc=0.4120
Epoch 700: loss=0.6690, acc=0.4630
Epoch 800: loss=0.6683, acc=0.5556
Epoch 900: loss=0.6658, acc=0.4676
Test Final del accuracy usando solo el precio: 0.5880


Finalmente, cuando se volvió a entrenar el LSTM utilizando solo la característica del precio, con los mismos hiperparámetros y estructura de red, la precisión final de la prueba fue de 0,5880. Esta precisión es muy similar al resultado obtenido cuando se incluyeron las ocho características[2].

Este resultado demuestra que la mayor parte de la señal predictiva capturada por el LSTM procedía directamente de la propia secuencia de precios sin procesar. La adición de otras variables, como las medias móviles, el impulso o el RSI, aportó poco poder predictivo adicional. Este hallazgo se ajusta al principio de que, en las series temporales financieras, muchos indicadores técnicos son transformaciones derivadas del precio, lo que introduce redundancia en lugar de nueva información.

Por ultimo, aunque la inclusión de múltiples características mejoró ligeramente la precisión, la diferencia fue marginal, lo que sugiere que el mecanismo de memoria interna del LSTM ya extrae gran parte de la información necesaria solo del historial de precios. Por lo tanto, la incorporación de variables adicionales podría no justificar la complejidad añadida del modelo, a menos que representen fuentes verdaderamente independientes de información predictiva.

**Referencias:**
[1] “Advanced stock market Prediction using Long Short-Term Memory Networks: A comprehensive deep learning framework.” https://arxiv.org/html/2505.05325v1

[2] A. Rahmadeyan and N. Mustakim, “Long Short-Term memory and gated recurrent unit for stock price prediction,” Procedia Computer Science, vol. 234, pp. 204–212, Jan. 2024, doi: 10.1016/j.procs.2024.02.167.

[3]P. Von Stackelberg, L. C. E. Huberts, R. Goedhart, and R. J. M. M. Does, “Multilevel model versus recurrent neural network: A case study to predict student success or failure revisited,” Journal of Quality Technology, pp. 1–20, Jan. 2025, doi: 10.1080/00224065.2024.2435870.

#El caso de mordeduras de serpientes